In [2]:
pip install mysql-connector-python psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
#libraries required
import pandas as pd
import mysql.connector
import pyodbc
import string
import json
import psycopg2


In [4]:
#Functions
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#library that contains punctuation
#defining the function to remove punctuation
def remove_punctuation_and_lower(text):
    punctuationfree="".join([i.lower() for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text



def load_json_data(input_file):
    try:
        with open(input_file, 'r') as data_json:
            source_data = json.load(data_json)
        return source_data
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
        return None

NameError: name 'recipes_with_ingredients' is not defined

In [5]:
# database = input("Enter database name:")
# host = input("Enter host name:")
# user = input("Enter user name:")
# password = input("Enter password:")
# port = input("Enter port number:")


cur = conn.cursor()

In [6]:
#product_ingredients_2018

list_of_tables = ['usda_2022_food_branded_experimental', 'usda_2022_branded_food_nutrients', 'usda_2022_nutrient_master']

# for i in list_of_tables:
#     cur.execute(f"SELECT * FROM {i} LIMIT 10")
#     records = cur.fetchall()
#     print([desc[0] for desc in cur.description])
#     cur.execute(f"SELECT COUNT({cur.description[0][0]}) FROM {i}")
#     count = cur.fetchall()
#     print(records)
#     print(count)

# cur.execute("SELECT DESCRIPTION FROM usda_2022_food_branded_experimental LIMIT 100")
# output = cur.fetchall()
# print(output)
#cur.execute("SELECT count(long_name), long_name FROM product_ingredients_2018 group by long_name LIMIT 5000")
#records = cur.fetchall()
#print([desc[0] for desc in cur.description])
#print(records)

cur.execute("""SELECT t1.fdc_id, t1.description, t2.nutrient_id, t2.amount, t3.name, t3.unit_name, t3.nutrient_nbr, t3.rank
            FROM usda_2022_food_branded_experimental 
            t1 LEFT JOIN usda_2022_branded_food_nutrients t2 ON t1.fdc_id = t2.fdc_id LEFT JOIN
            usda_2022_nutrient_master t3 ON t2.nutrient_id = t3.id LIMIT 100""")
join_records = cur.fetchall()
columns = [desc[0] for desc in cur.description]
join_df = pd.DataFrame(join_records, columns=columns)

join_df

# pd.set_option('display.max_colwidth', None)

group_object = join_df.groupby(['description'])['nutrient_nbr'].mean().reset_index()

join_df.description.iloc[:10]
#skip the experimental

0    ZITI PARMESANO
1    ZITI PARMESANO
2    ZITI PARMESANO
3    ZITI PARMESANO
4    ZITI PARMESANO
5    ZITI PARMESANO
6    ZITI PARMESANO
7    ZITI PARMESANO
8    ZITI PARMESANO
9    ZITI PARMESANO
Name: description, dtype: object

In [9]:
pd.set_option('display.max_colwidth', None)
group_object.description



0    6 CHEESE ITALIAN LOW MOISTURE PART-SKIM MOZZARELLA, PROVOLONE WITH SMOKE FLAVOR, ROMANO, FONTINA, FRESH ASIAGO & PARMESAN FINE CUT SHREDDED NATURAL CHEESES, 6 CHEESE ITALIAN
1                                                                                                                           BLUEBERRY GREEK YOGURT MEAL REPLACEMENT BAR, BLUEBERRY
2                                                                                                                           COOKIES & CREME PROTEIN-RICH MEAL BAR, COOKIES & CREME
3                                                                                               DARK CHOCOLATE TRUFFLE CASHEWMILK NON-DAIRY FROZEN DESSERT, DARK CHOCOLATE TRUFFLE
4                                                                 TURKEY BREAST MEDALLIONS & STUFFING WITH HOME STYLE MASHED POTATOES & GRAVY, TURKEY BREAST MEDALLIONS & STUFFING
5                                                                                                        

In [42]:
# table_type_column_query = f"""SELECT column_name, data_type 
#                              FROM information_schema.columns
#                              WHERE table_name = '{tbl}';"""

for tbl in list_of_tables:
    table_type_column_query = f"""SELECT column_name, data_type 
                             FROM information_schema.columns
                             WHERE table_name = '{tbl}';"""
    cur.execute(table_type_column_query)
    records = cur.fetchall()
    print('\n')
    print(tbl)
    print(pd.DataFrame(records))




usda_2022_food_branded_experimental
                  0        1
0            fdc_id  integer
1         data_type     text
2       description     text
3  food_category_id     text
4  publication_date     text


usda_2022_branded_food_nutrients
                    0        1
0                  id  integer
1              fdc_id  integer
2         nutrient_id  integer
3              amount  numeric
4         data_points     text
5       derivation_id  integer
6                 min     text
7                 max     text
8              median     text
9            footnote     text
10  min_year_acquired     text


usda_2022_nutrient_master
              0        1
0            id  integer
1          name     text
2     unit_name     text
3  nutrient_nbr  numeric
4          rank     text


In [44]:
#prof deleted this table -- probably not good
foods_to_avoid = pd.read_csv('Disease_foods_to_avoid_csv.csv', delimiter = ';')
#load in tasty dishes (csv)
tasty_dishes = pd.read_csv('dishes.csv') 
#load in tasty ingredients and instructions as Pandas dataframe
with open('ingredient_and_instructions.json') as file:
    tasty_json_ingredients = json.load(file)
ingredients_and_recipes_df = pd.DataFrame(tasty_json_ingredients)


In [52]:
#transpose and print a test
df = ingredients_and_recipes_df.transpose()
df.iloc[0,0]
#Nodes: Recipe, Ingredient
#Relationships: Ingredients -> Ingredients via instructions


[{'name': 'Dough',
  'ingredients': [{'extra_comment': 'melted',
    'name': 'unsalted butter',
    'primary_unit': {'quantity': '½', 'display': 'cup'},
    'metric_unit': {'quantity': '115', 'display': 'g'}},
   {'extra_comment': 'warm to the touch',
    'name': 'whole milk',
    'primary_unit': {'quantity': '2', 'display': 'cups'},
    'metric_unit': {'quantity': '480', 'display': 'mL'}},
   {'extra_comment': '',
    'name': 'granulated sugar',
    'primary_unit': {'quantity': '½', 'display': 'cup'},
    'metric_unit': {'quantity': '100', 'display': 'g'}},
   {'extra_comment': '',
    'name': 'active dry yeast',
    'primary_unit': {'quantity': '2 ¼', 'display': 'teaspoons'},
    'metric_unit': None},
   {'extra_comment': 'divided',
    'name': 'flour',
    'primary_unit': {'quantity': '5', 'display': 'cups'},
    'metric_unit': {'quantity': '625', 'display': 'g'}},
   {'extra_comment': '',
    'name': 'baking powder',
    'primary_unit': {'quantity': '1', 'display': 'teaspoon'},
   

In [105]:
tasty_dishes_filtered = tasty_dishes[['country', 'id_', 'description', 'keywords', 'name', 'cook_time', 'prep_time', 'total_time',
       'ratings_negative', 'ratings_positive', 'score', 'protein', 'fat',
       'calories', 'sugar', 'carbohydrates', 'fiber']]

tasty_dishes_dropped_nas = tasty_dishes_filtered.dropna(subset=['score'])
print(tasty_dishes_dropped_nas.dtypes)

tasty_dishes_dropped_nas.iloc[2967,:]
#tasty_dishes_dropped_nas['description'] = tasty_dishes_dropped_nas['description'].astype('string').apply(lambda x: remove_punctuation(x))

#tasty_dishes_dropped_nas['keyword'] = tasty_dishes_dropped_nas['keywords'].str.replace(/\"(.*?)\"/g, "{$1}")

tasty_dishes_test_csv = tasty_dishes_dropped_nas.to_csv(index=False)
print(tasty_dishes_test_csv[2500:3000])

tasty_dishes_dropped_nas.to_csv('tasty_dishes_cleaned.csv', index=False)


country              object
id_                   int64
description          object
keywords             object
name                 object
cook_time             int64
prep_time             int64
total_time            int64
ratings_negative      int64
ratings_positive      int64
score               float64
protein             float64
fat                 float64
calories            float64
sugar               float64
carbohydrates       float64
fiber               float64
dtype: object
 For Breakfast,0,0,0,133,2621,96.0,13.0,19.0,242.0,0.0,3.0,0.0
US,3033,,,3-ingredient Pancakes,0,0,0,533,2947,85.0,4.0,3.0,80.0,4.0,9.0,1.0
US,1352,,,Chocolate Strawberry Crepes ,0,0,0,143,2779,96.0,41.0,56.0,1409.0,66.0,181.0,9.0
ZZ,1477,,"breakfast, healthy, oats, pancakes",Banana Pancakes,0,0,0,773,2694,78.0,4.0,3.0,116.0,9.0,18.0,2.0
ZZ,2267,,"buzzfeed, cake, crepe, dessert, rainbow, tasty, tasty_vegetarian",Rainbow Crepe Cake,0,0,0,294,1889,87.0,10.0,18.0,351.0,12.0,34.0,0.0
US,1630,,"
